In [1]:
from transformers import pipeline
import pandas as pd

/home/francisco/Documents/Francisco/Faculdade/7_Semestre/processamento_lingua_natural/tpFinal/tp_final_nlp/env_py/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-25 17:16:13.034958: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 17:16:13.341851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737836173.461298   14685 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one h

In [2]:
# load the data
data = pd.read_csv('./data/anime-dataset-2023.csv')


In [4]:
# see column names
print(data.columns)

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')


In [5]:
data.head(2)

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...


In [3]:
# get only the useful columns -> 'anime_id', 'Name', 'Genres', 'Synopsis'
data = data[['anime_id', 'Name', 'Genres', 'Synopsis']]

data.head()

,anime_id,Name,Genres,Synopsis
0,1,Cowboy Bebop,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi","Another day, another bounty—such is the life o..."
2,6,Trigun,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...
4,8,Bouken Ou Beet,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...


In [4]:
# split data into 2 parts -> train and test
# train -> 80% of the data
# test -> 20% of the data
train = data.sample(frac=0.8, random_state=0)
test = data.drop(train.index)

In [8]:
data.shape, train.shape, test.shape

((24905, 4), (19924, 4), (4981, 4))

In [5]:
# predict the genre of the anime

# get the first anime synopsis
anime_synopsis = test['Synopsis'].iloc[0]
anime_name = test['Name'].iloc[0]

# all possible genres
all_genres = list(set(data['Genres']))
genres = set()
for i in all_genres:
    for j in i.split(','):
        cat = j.strip().lower()
        # remove ' in cat
        cat = cat.replace("'", "")
        genres.add(cat)

print(genres)

{'sports', 'award winning', 'fantasy', 'sci-fi', 'slice of life', 'unknown', 'adventure', 'avant garde', 'suspense', 'boys love', 'supernatural', 'action', 'romance', 'ecchi', 'drama', 'hentai', 'mystery', 'girls love', 'erotica', 'comedy', 'horror', 'gourmet'}


In [20]:
print(anime_synopsis)

Moments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrous beast inside the newborn Naruto.

Now, Naruto is a hyperactive and knuckle-headed ninja still living in Konohagakure. Shunned because of the Kyuubi inside him, Naruto struggles to find his place in the village, while his burning desire to become the Hokage of Konohagakure leads him not only to some great new friends, but also some deadly foes.


In [7]:

labels = list(genres)

# Create a zero-shot classification pipeline using Hugging Face
classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-base")

# Text to classify
text = anime_synopsis

# Batch labels into groups of 10
batch_size = 10
related_categories = {}

for i in range(0, len(labels), batch_size):
    batch_labels = labels[i:i + batch_size]
    result = classifier(text, batch_labels)
    for label, score in zip(result["labels"], result["scores"]):
        related_categories[label] = score

# Sort the related categories by score
sorted_categories = sorted(related_categories.items(), key=lambda x: x[1], reverse=True)

# Display the results
for label, score in sorted_categories:
    print(f"{label}: {score:.4f}")


ValueError: Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast convertors: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'RealmTokenizer', 'Refor